In [1]:
#Pretty Display of Variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#%autoreload

# Import libraries

In [3]:
from plot_projections import plot_all_projections, plot_final_projections, plot_point_for_couple, plot_projection_up_down, plot_border
from prop_info import extreme_points, vect_blade, d_blade, center_prop
from get_segments import blade_alone, get_segments_points, get_planes
from major_axis import get_major_axis
#from projections import couple_all_planes, project_all_couples, projections_by_side, project_couple
from new_projections import assign_points, get_all_points_for_projections, interpolations, find_separation_plane, interpolate_points, model_func, points_from_curve
from parameters import get_hub_points, get_hub_radius
from plot_param import plot_hub
from plot_prop import plot_pointcloud, plot_direction, plot_segments
from myMathFunction import least_squares, point_on_plane
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# Pre-processing

In [4]:
propeller_coords = pd.read_csv('aerostar_data.csv')
propeller_coords.shape

(151986, 3)

In [5]:
propeller_coords = propeller_coords.drop_duplicates(subset=None, keep='first', inplace=False)
propeller_coords = propeller_coords.reset_index(drop=True)

In [6]:
max_point, min_point, middle_point, highest_point, lowest_point = extreme_points(propeller_coords)

vect_length = vect_blade(max_point, min_point) 

dmiddle, dhighest, dlowest = d_blade(vect_length, middle_point, highest_point, lowest_point)

upper_blade, lower_blade = blade_alone(propeller_coords, vect_length, dmiddle)

vect_out, vect_side, hub_inner_radius = get_major_axis(propeller_coords, middle_point, vect_length)

# Projections

In [7]:
nb_seg = 3
resolution = 3
nb_points = 100

planes = get_planes(upper_blade, dmiddle, dhighest, vect_length, nb_seg)
segments = get_segments_points(upper_blade, planes, nb_seg)
#plot_segments(segments)

up, down = get_all_points_for_projections(planes, segments, nb_seg, resolution)

In [8]:
up1 = up[0]
dn1 = down[0]
plan1 = planes[0]
#print(up1.shape)

In [9]:
# 1. Find border points
up_side1_border, up_side2_border, _, _, _ = extreme_points(up1)
dn_side1_border, dn_side2_border, _, _, _ = extreme_points(dn1)
#print("up_right {}\n".format(up_right))

In [10]:
# 2. Find separating plane
C_up = find_separation_plane(up1.values)
C_dn = find_separation_plane(dn1.values)
# Z = C[4]*X**2. + C[5]*Y**2. + C[3]*X*Y + C[1]*X + C[2]*Y + C[0]
#print("C_up {}\n".format(C_up))

In [11]:
# 3. Assign point to side  (do it for both sides on both sides)
up_right_points, up_left_points = assign_points(C_up, up1)
dn_right_points, dn_left_points = assign_points(C_dn, dn1)
#print("right_points_up_shape {}\n".format(up_right_points.shape))

In [12]:
# Add border points to fit
up_right_points = up_right_points.append(pd.DataFrame(up_side1_border.reshape(1, 3), columns = ["X","Y","Z"]))
up_right_points = up_right_points.append(pd.DataFrame(up_side2_border.reshape(1, 3), columns = ["X","Y","Z"]))
#print("up_right_points_shape {}\n".format(up_right_points.shape))

In [21]:
from scipy.optimize import curve_fit
def interpolate_points(up1):
    data = np.c_[up1.values[:,0], up1.values[:,1]]
    z = up1.values[:,2]

    sigma = np.ones(len(data))
    sigma[[-1, -2]] = 0.1  #assign more weight to border points
    popt, pcov = curve_fit(model_func, data, z, sigma=sigma)    
    return popt

In [22]:
# 4. Interpolate points
up_right_popt = interpolate_points(up_right_points)
up_left_popt = interpolate_points(up_left_points)
dn_right_popt = interpolate_points(dn_right_points)
dn_left_popt = interpolate_points(dn_left_points)

In [24]:
# 5. Final projection
# Take points on each side
up_right_pts = points_from_curve(up_side1_border, up_side2_border, nb_points, up_right_popt)
dn_right_pts = points_from_curve(dn_side1_border, dn_side2_border, nb_points, dn_right_popt)

up_left_pts = points_from_curve(up_side1_border, up_side2_border, nb_points, up_left_popt) 
dn_left_pts = points_from_curve(dn_side1_border, dn_side2_border, nb_points, dn_left_popt)

In [36]:
# Projection de la ligne reliant 2 points sur le plan
proj_right = []
proj_left = []
labels = ['X', 'Y', 'Z']
for i in range(0, up_right_pts.shape[0]):
    proj_right.append(point_on_plane(up_right_pts[i], dn_right_pts[i], plan1))
    proj_left.append(point_on_plane(up_left_pts[i], dn_left_pts[i], plan1))
proj_right_df = pd.DataFrame(proj_right, columns = labels)
proj_left_df = pd.DataFrame(proj_left, columns = labels)

In [33]:
projection_right = interpolate_points(proj_right_df)

In [37]:
proj_right_df

,X,Y,Z
0,0.036879,12.997339,-11.267194
1,0.037288,12.870880,-11.038369
2,0.037690,12.744380,-10.814061
3,0.038085,12.617839,-10.594228
4,0.038473,12.491255,-10.378826
5,0.038855,12.364630,-10.167812
6,0.039229,12.237963,-9.961143
7,0.039597,12.111253,-9.758777
8,0.039959,11.984501,-9.560669
9,0.040314,11.857707,-9.366777


# Find parameters

    - Tip radius 
        Projection dans le sens de la longueur et ... ??
    - Blade twist 
        Project on planes as in projection
        Compute difference of angle between main directions of projections (PCA possible)

### Hub radius

In [ ]:
hub_points = get_hub_points(propeller_coords, dmiddle, vect_length)

hub_outer_radius, hub_inner_radius = get_hub_radius(hub_points, middle_point, vect_side)

hub_radius = hub_outer_radius[2] - middle_point[2]  #from center to exterior radius

In [ ]:
plot_hub(propeller_coords, hub_points, hub_outer_radius, hub_inner_radius)

### Chord length

In [ ]:
chord_length = np.linalg.norm(highest_point - middle_point) - hub_radius
chord_length

### Blade twist

### Tip radius